In [1]:
import pandas as pd
import yaml

with open("./local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

# paths to import files
schematic_config = config["paths"]["schematic"]
csv_model = config["file_names"]["csv_model"]
json_model = config["file_names"]["json_model"]

print(
    "Schematic config: ",
    schematic_config,
    "\n",
    "CSV model: ",
    csv_model,
    "\n",
    "JSON LD Model: ",
    json_model,
)


Schematic config:  ./config.yml 
 CSV model:  EL.data.model.csv 
 JSON LD Model:  EL.data.model.jsonld


In [8]:
print(f'schematic schema convert ../{csv_model} --output_jsonld ../{json_model}')

!schematic schema convert ../{csv_model} --output_jsonld ../{json_model}

schematic schema convert ../EL.data.model.csv --output_jsonld ../EL.data.model.jsonld
Starting schematic...
^C

Aborted!


## Get manifest names to generate manifests


In [ ]:
import json

with open(json_model, "r") as jf:
    jo = json.load(jf)

In [ ]:
# Manifest names in data model
manifest_names_extracted = []

for i in jo["@graph"]:
    if i["@id"] == "bts:dataType":
        manifest_names_extracted.append(
            i["schema:domainIncludes"]["@id"].replace("bts:", "")
        )

# display names extracted
manifest_display_names_extracted = []

for i in jo["@graph"]:
    if i["@id"].replace("bts:", "") in (manifest_names_extracted):
        manifest_display_names_extracted.append(i["sms:displayName"])
manifest_display_names_extracted

# Create dictionary for lookup later
manifest_name_relationships = dict(
    zip(manifest_names_extracted, manifest_display_names_extracted)
)

manifest_name_relationships

In [ ]:
# Create the manifest templates based on the tables
manifests = (
    dm2.query('Properties == "dataProperty"')
    .groupby("Parent")
    .agg({"Attribute": list})
    .reset_index()
)

manifests["Attribute"] = manifests["Attribute"].apply(lambda x: x + ["Component"])

manifests["Required"] = "True"
manifests["Properties"] = "dataType"

manifests = manifests.rename({"Attribute": "DependsOn", "Parent": "Attribute"}, axis=1)
manifests["DependsOn"] = manifests["DependsOn"].apply(lambda x: ",".join(x))


# Store the data types in a list
dataTypes = manifests["Attribute"].tolist()

In [ ]:
# Template configuration


def manifest_template(k, v, t="file"):
    manifest_template = {"display_name": v, "schema_name": k, "type": t}
    return manifest_template


dca_template = {
    "manifest_schemas": [],
    "service_version": "v23.1.1",
    "schema_version": "v1.2",
}

records = [
    "IndividualHuman",
    "IndividualnonHuman",
    "Biospecimenhuman",
    "BiospecimennonHuman",
]


for k, v in manifest_name_relationships.items():
    if k in records:
        t = "records"
    else:
        t = "file"
    dca_template["manifest_schemas"].append(manifest_template(k, v, t))

json_formatted_str = json.dumps(dca_template, indent=2)
print(json_formatted_str)

with open(
    "../_data/dca_template.json",
    "w",
) as f:
    f.write(json_formatted_str)

## Generate Manifests


In [ ]:
manifest = 'IndividualHuman'

print(f"schematic manifest --config {schematic_config} get -dt {manifest} --output_csv {os.path.join(output_path, manifest + '.csv')} --title {'EL_Manifest_' + manifest} --sheet_url 2>&1 >> manifest_generation_results.txt")

!echo '{manifest}:' >> manifest_generation_results.txt
!schematic manifest --config {schematic_config} get -dt {manifest} --output_csv {os.path.join(output_path, manifest + '.xlsx')} --title {'EL_Manifest_' + manifest} --sheet_url 2>&1 >> manifest_generation_results.txt

In [ ]:
!echo ----- newly_generated_manifests ------- 2>&1 >> manifest_generation_links.txt
for manifest in manifest_names_extracted:
    print(manifest)
    !echo {manifest}: >> manifest_generation_results.txt
    !schematic manifest --config {schematic_config} get -dt {manifest} --output_xlsx {os.path.join(output_path, manifest + '.xlsx')} --title {'EL_Manifest_' + manifest} --sheet_url 2>&1 >> manifest_generation_results.txt
    !echo ----------------------------------------------------------------